In [5]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])
_.names

('c',)

### 3.4 命名张量

In [6]:
img_t = torch.randn(3, 5, 5) # 形状[通道，行，列] 
weights = torch.tensor([0.2126, 0.7152, 0.0722])
img_t.shape

torch.Size([3, 5, 5])

In [9]:
batch_t = torch.randn(2, 3, 5, 5) # 形状[批，通道，行，列]
batch_t.shape

torch.Size([2, 3, 5, 5])

In [8]:
img_gray_naive = img_t.mean(-3) #倒数第三个维度 求平均
batch_gray_naive = batch_t.mean(-3)

In [15]:
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

> weights.unsqueeze(-1)：在倒数第一个维度上再添加一个维度，unsqueeze_()这个函数和unsqueeze的功能是一样的，但是unsqueeze_()直接改变原输入张量，不带下划线的不会改变原输入张量。

In [16]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [17]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [18]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns') # refine把张量的每个维度重新定义名称
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [19]:
weights_aligned = weights_named.align_as(img_named) # a.align_as(b)：按照输入张量b的维度，补充a缺少的维度 自动对齐名称
weights_aligned.shape, weights_aligned.names 

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [20]:
gray_named = (img_named * weights_aligned).sum('channels') #-3 -> channelas
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

> 当我们尝试将不同名称的放在一起操作，是会发生错误的

In [21]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [22]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))